Processed data from 23/03/2025 to 18/09/2025

In [1]:
import pandas as pd
path=pd.read_csv('/home/hieu/Work/new_casacom/result/path_7.csv')
home=pd.read_csv('/home/hieu/Work/new_casacom/result/home_7.csv')
work=pd.read_csv('/home/hieu/Work/new_casacom/result/work_7.csv')
leisure=pd.read_csv('/home/hieu/Work/new_casacom/result/leisure_7.csv')


In [2]:
home.shape,work.shape,leisure.shape,path.shape

((357825, 78), (221989, 78), (352657, 78), (238850, 9))

In [13]:
path_sample=path.head().reset_index(drop=True)
path_sample.loc[0]['coordinate']

'[(33.55726357945241, -7.600792331417324), (33.55833625793457, -7.600893974304199), (33.55991897583008, -7.600734257698059), (33.5663948059082, -7.591155052185059), (33.57048649787903, -7.589333385229111), (33.56571006774902, -7.590679049491882), (33.56354514035311, -7.600614417682994), (33.56195002794266, -7.601624567061663), (33.56517028808594, -7.5920257568359375), (33.560098103114534, -7.5988949707576205), (33.55897521972656, -7.599021911621094), (33.591809231894366, -7.615708991459438), (33.58175500233968, -7.602296710014343), (33.58549690246582, -7.596461057662964), (33.568027843128554, -7.595923605290326), (33.58366394042969, -7.596599102020264), (33.561608592669174, -7.600298675894737), (33.56856155395508, -7.591278076171875), (33.56234752425361, -7.60055389809532), (33.58488845825195, -7.597406387329102), (33.58622360229492, -7.595643043518066), (33.573019206523895, -7.59370132163167), (33.5708311535418, -7.596536149503663), (33.557945251464844, -7.586170196533203), (33.570789

In [ ]:
import plotly.express as px
import ast

# Convert string representation of coordinates to actual list of tuples
path_sample['coord_list'] = path_sample['coordinate'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Create lists for latitudes and longitudes
all_lats = []
all_lons = []
maid_ids = []

# Extract coordinates for each path
for idx, row in path_sample.iterrows():
    coords = row['coord_list']
    lats = [coord[0] for coord in coords]
    lons = [coord[1] for coord in coords]
    
    # Add to our lists
    all_lats.extend(lats)
    all_lons.extend(lons)
    maid_ids.extend([row['maid'][:10] + '...' for _ in range(len(coords))])

# Create a dataframe for plotting
plot_df = pd.DataFrame({
    'latitude': all_lats,
    'longitude': all_lons,
    'maid': maid_ids
})

# Create the map
fig = px.scatter_mapbox(plot_df, 
                        lat='latitude', 
                        lon='longitude', 
                        color='maid',
                        zoom=10,
                        mapbox_style="open-street-map",
                        title="Path Coordinates by MAID")

fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

# Display the original dataframe as well
path_sample

In [4]:
# Calculate the number of unique MAIDs across all datasets
unique_maids = set().union(
    home.maid.unique(), 
    path.maid.unique(), 
    work.maid.unique(), 
    leisure.maid.unique()
)

# Print the count of unique MAIDs
print(f"Number of unique MAIDs processed: {len(unique_maids)}")


Number of unique MAIDs processed: 684892


In [5]:
print(f"Total home points: {home.shape[0]}")
print(f"Total work points: {work.shape[0]}")
print(f"Total leisure points: {leisure.shape[0]}")
print(f"Total path points: {path.shape[0]}")
print(f"Total combined: {home.shape[0] + work.shape[0] + leisure.shape[0] + path.shape[0]}")

Total home points: 357825
Total work points: 221989
Total leisure points: 352657
Total path points: 238850
Total combined: 1171321


In [6]:
home_p,work_p,leisure_p=home[home.country=='Morocco'].shape[0]/home.shape[0]*100,work[work.country=='Morocco'].shape[0]/work.shape[0]*100,leisure[leisure.country=='Morocco'].shape[0]/leisure.shape[0]*100
print(f"Percentage of home points in Morocco: {home_p:.2f}%")
print(f"Percentage of work points in Morocco: {work_p:.2f}%")
print(f"Percentage of leisure points in Morocco: {leisure_p:.2f}%")


Percentage of home points in Morocco: 71.87%
Percentage of work points in Morocco: 69.04%
Percentage of leisure points in Morocco: 77.40%


In [7]:
# Calculate percentage of points in countries other than Morocco
other_home_p = 100 - home_p
other_work_p = 100 - work_p
other_leisure_p = 100 - leisure_p

print(f"Percentage of home points in other countries: {other_home_p:.2f}%")
print(f"Percentage of work points in other countries: {other_work_p:.2f}%")
print(f"Percentage of leisure points in other countries: {other_leisure_p:.2f}%")


Percentage of home points in other countries: 28.13%
Percentage of work points in other countries: 30.96%
Percentage of leisure points in other countries: 22.60%


In [8]:
df=pd.concat([home,work,leisure])
df.reset_index(drop=True,inplace=True)
fluxes_pings=df.groupby('maid').agg({
    "maid_fluxB":"first",
    "maid_fluxC":"first",
    "maid_fluxD":"first",
    "maid_fluxE":"first",
    "maid_fluxF":"first",
})
b_pings=fluxes_pings.maid_fluxB.sum()
c_pings=fluxes_pings.maid_fluxC.sum()
d_pings=fluxes_pings.maid_fluxD.sum()
e_pings=fluxes_pings.maid_fluxE.sum()
f_pings=fluxes_pings.maid_fluxF.sum()
total_pings=b_pings+c_pings+d_pings+e_pings+f_pings
b_percent = b_pings/total_pings*100
c_percent = c_pings/total_pings*100
d_percent = d_pings/total_pings*100
e_percent = e_pings/total_pings*100
f_percent = f_pings/total_pings*100
# Display results
print(f"Flux B: {round(b_percent,2)}%")
print(f"Flux C: {round(c_percent,2)}%")
print(f"Flux D: {round(d_percent,2)}%")
print(f"Flux E: {round(e_percent,2)}%")
print(f"Flux F: {round(f_percent,2)}%")

Flux B: 64.27%
Flux C: 22.33%
Flux D: 11.44%
Flux E: 0.53%
Flux F: 1.43%


In [16]:
sample_home=home[:20]
sample_work=work[:20]

In [27]:
sample_home.keys()

Index(['maid', 'geohash', 'category', 'confidence', 'country', 'lat', 'lon',
       'first_seen', 'last_seen', 'est_duration', 'pings', 'unique_days',
       'evidence_score', 'fluxB', 'fluxC', 'fluxD', 'fluxE', 'fluxF', 'monday',
       'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'poi_score',
       'poi_info', 'home', 'work', 'leisure', 'path', 'maid_total_pings',
       'cell_info', 'maid_fluxB', 'maid_fluxC', 'maid_fluxD', 'maid_fluxE',
       'maid_fluxF', 'maid_countries', 'maid_countries_pings',
       'maid_first_seen', 'maid_last_

In [ ]:
pd.set_option('display.max_columns', None)
sample_work['poi_info']

In [23]:
import pygeohash as pgh
pgh.geohash_haversine_distance('evdk4h3','evdk4h2')

129.97087482528673

In [19]:
# Create a scatter plot of sample home and work locations using plotly with OSM tiles
import plotly.graph_objects as go
import plotly.express as px

# Create figure with OpenStreetMap tiles
fig = go.Figure()

# Add home points
fig.add_trace(go.Scattermapbox(
    lon=sample_home['lon'],
    lat=sample_home['lat'],
    mode='markers',
    marker=dict(
        size=10,
        color='blue',
        opacity=0.7
    ),
    text=sample_home['confidence'],
    hoverinfo='text',
    name='Home'
))

# Add work points
fig.add_trace(go.Scattermapbox(
    lon=sample_work['lon'],
    lat=sample_work['lat'],
    mode='markers',
    marker=dict(
        size=10,
        color='red',
        opacity=0.7
    ),
    text=sample_work['confidence'],
    hoverinfo='text',
    name='Work'
))

# Update layout to use OpenStreetMap
fig.update_layout(
    title='Sample Home and Work Locations',
    mapbox=dict(
        style='open-street-map',
        zoom=10,
        center=dict(
            lat=sample_home['lat'].mean(),
            lon=sample_home['lon'].mean()
        )
    ),
    height=600,
    width=800,
    legend=dict(
        x=0,
        y=0,
        orientation='h'
    )
)

fig.show()


/tmp/ipykernel_1423418/4101926460.py:9: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/tmp/ipykernel_1423418/4101926460.py:24: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

